# Offline Learning Resources for Comprehensive Project

## Load libraries and configure environment

In [1]:
%load_ext autoreload
%autoreload 2

import yaml
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

from data.imagenet_loader import ContinualImageNet
from models.backbone import get_backbone
from strategies.join_trainning import JoinTrainer
from utils.metrics import CLMetrics

# Cấu hình hiển thị
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 6)

ModuleNotFoundError: No module named 'strategies.join_trainning'

In [ ]:
# 1. Khởi tạo Dataset Manager
cl_dataset = ContinualImageNet(
    root_dir=cfg['data']['root_dir'], 
    num_tasks=cfg['data']['num_tasks'], 
    total_classes=cfg['data']['total_classes'],
    batch_size=cfg['training']['batch_size'],
    num_workers=cfg['data']['num_workers']
)

# 2. Khởi tạo Metrics Tracker
# Đây là công cụ quan trọng để tính Forgetting Measure
cl_metrics = CLMetrics(cfg['data']['num_tasks'])

# Danh sách lưu lịch sử để vẽ biểu đồ theo thời gian
history = {
    'avg_acc': [],
    'avg_forget': [],
    'task_acc': {i: [] for i in range(cfg['data']['num_tasks'])}
}